In [7]:
from pptx import Presentation
import pandas as pd
import numpy as np
from pydataset import data
import plotlyPowerpoint as pp
import warnings
warnings.filterwarnings('ignore')

############
## Prepare Data
############

#load datasets
global df
df = data('InsectSprays')
global df2
df2 = data("JohnsonJohnson")

#Data transformation
df['m2'] = df['count'] * 1.1
df2['year'] = df2['time'].astype(int)
df2 = df2.groupby(['year']).agg({'JohnsonJohnson': 'mean'}).reset_index()

#form df3 to showcase faceting on line chart
temp = df2.copy()
temp['company'] = 'J&J'
temp = temp.rename(columns={'JohnsonJohnson': 'price'})

temp2 = temp.copy()
temp2['company'] = 'Phizer'
temp2['price'] = temp2['price'] * 1.1

df3 = pd.concat([temp, temp2])

#form df4 to showcase faceting and grouping by additional dimension
df4 = df.copy()
df4['category'] = df['spray']
df4.loc[df4['category'].isin(['A','B','C']), 'category'] = 'A'
df4.loc[df4['category'].isin(['D','E','F']), 'category'] = 'B'

#form df for showcasing a table
df5 = df.head()
df5['m2'][2] = df5['m2'][2] / 2
df5['m2'][3] = None #showing how the library handles NaN values
df5['m3'] = df5['m2'] / 100
df5['m4'] = df5['m2']

#setup color map for table
colorDf = df5.copy()
colorDf['m2'] = np.select(
    [
        colorDf['m2'] >= colorDf['count'],
        colorDf['m2'] < colorDf['count']
    ],
    [
        "#5eed4e",
        "#ed4e61"
    ]
)

colorDf['count'] = '#fffff2'
colorDf['spray'] = '#fffff2'
colorDf['m3'] = '#fffff2'
colorDf['m4'] = '#fffff2'

In [8]:
############
## Setup Variableas
############
import plotly.express as px

#define custom color palette
# https://plotly.com/python/discrete-color/
colors = px.colors.qualitative.Vivid

#set template for funciton
pp.setTemplate("template.pptx")

#set color palette
pp.setColors(colors)

#setup each slide
charts = [
    { #Line Chart - stock prices
        "data": df2,
        "type": "line",
        "name": "Stock Prices by Company",
        "filename": 'charts/stock-prices-by-company',
        "metrics": [
            {"name": "JohnsonJohnson", "prettyName": "Stock Price", "method": "mean"}
        ],
        "axis": "year",
        "x-axis-title": 'Year',
        "y-axis-title": "Average Stock Price",
        "description": "Grouping by additional variables is easy",
        "size": "wide",
        "filters": [
            {"variable": "year", "operation": ">=", "value": "1970", "type":"int"}
        ],
        "item-index": {
            'slide': 0,
            'title': 0,
            'chart': 10,
            'description': 11
        }
    },
    { #Bar chart of insect sprays
        "data": df4,
        "type": "bar",
        "name": "Avg Spray Effictiveness by Type",
        "filename": 'charts/spray-by-type',
        "metrics": [
            {"name": "count", "prettyName": "Effectiveness", "method": "mean"},
            {"name": "m2", "prettyName": "Effectiveness 2", "method": "mean"}
        ],
        "axis": "spray",
        "color": "category", 
        "x-axis-title": "Effectiveness",
        "description": "this slide has data on it!",
        'options': {
            'orientation': 'horizontal'
        },
         "item-index": {
            'slide': 0,
            'title': 0,
            'chart': 10,
            'description': 11
        }
    },
    { # Facet Line Chart
        "data": df3,
        "type": "facetLine",
        "name": "Multiple Comanpy Stock Prices",
        "filename": 'charts/many-stock-prices-facet',
        "metrics": [
            {"name": "price", "prettyName": "Stock Price", "method": "mean"}
        ],
        "axis": "year",
        'y-axis-title': 'Stuff',
        "description": "Faceting can be fun",
        "filters": [
            {"variable": "year", "operation": ">=", "value": "1970", "type":"int"}
        ],
        'facet': 'company',
        'facet-direction': 'rows',
        "options": {
            'horizontal-grid-lines': 'true',
            'vertical-grid-lines': 'true',
            'color-grouping': 'facet',
            'facet-spacing': 0.3
        },
         "item-index": {
            'slide': 0,
            'title': 0,
            'chart': 10,
            'description': 11
        }
    },
    { # Facet Bar chart
        "data": df4,
        "type": "facetBar",
        "name": "Avg Spray Effictiveness by Type",
        "filename": 'charts/spray-by-type',
        "metrics": [
            {"name": "count", "prettyName": "Effectiveness", "method": "mean"}
        ],
        "axis": "spray",
        "facet": 'category',
        "facet-direction": 'rows',
        "x-axis-title": "Effectiveness",
        "description": "this slide has data on it!",
        "options": {
            'color-grouping': 'facet'
        },
        "item-index": {
            'slide': 0,
            'title': 0,
            'chart': 10,
            'description': 11
        }
        
    },
    { # Table Test
        "data": df5,
        "type": "table",
        "name": "Example of a Table",
        "fill_color": colorDf,
        "column_formats": ['number', 'string', 'money', 'percent', 'twoDigitNum'],
         "item-index": {
            'slide': 2,
            'title': 0,
            'chart': 12,
            'description': 11
        },
        'header_fill_color': '#051e4a',
        'header_text_color': '#ffffff',
        'text_color': '#051e4a'
    }
]


In [9]:
############
## Run Function
############
pp.createSlides(charts)

ValueError: cannot convert float NaN to integer

In [23]:
df5

,count,spray,m2,m3,m4
1,10,A,11.00,0.1100,11.00
2,7,A,3.85,0.0385,3.85
3,20,A,NaN,NaN,NaN
4,14,A,15.40,0.1540,15.40
5,14,A,15.40,0.1540,15.40


In [22]:
#for loop for the rows
for i in range(len(df5) + 1):
    #for each row, get the value of the column
    for i2 in range(len(df5.columns)):

        #if we're dealing with the header
        if i == 0:
            nothing = 'nothing'
        else:
            text = df5.iloc[i-1, i2]
            textFormat = charts[4]['column_formats'][i2]

            #catch Nan values for numeric based values
            if 'float' in str(type(text)) or 'int' in str(type(text)):
                if math.isnan(text):
                    print("found NaN in num col")
                    continue

            #catch Nan values for string based values
            if 'str' in str(type(float)) or 'NoneType' in str(type(text)):
                if text is None:
                    print("found NaN in string col")
                    continue

            if textFormat == 'number':
                nothing = 'nothing'
            elif textFormat == 'money':
                print('$' + str(int(text)))
            elif textFormat == 'percent':
                nothing = 'nothing'
            elif textFormat == 'twoDigitNum':
                nothing = 'nothing'
            elif textFormat == 'date':
                nothing = 'nothing'
            else:
                nothing = 'nothing'

$11
$3
found NaN in num col
found NaN in num col
found NaN in num col
$15
$15
